In [1]:
from utils import *
import numpy as np
import torch, os
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
set_random_seed(42)

Seed: 42


In [3]:
# def scaling_embeddings(embeddings):
#     for i in range(len(embeddings)):
#         embeddings[i] = StandardScaler().fit_transform(embeddings[i])
#     return embeddings

def reshape_embeddings(embeddings):
    if embeddings.shape[1] == 1:
        embeddings = embeddings.squeeze(1)
    return embeddings

In [4]:
dynahate_labels_train = process_labels(read_labels("dynahate", "train"))
dynahate_labels_dev = process_labels(read_labels("dynahate", "dev"))
dynahate_labels_test = process_labels(read_labels("dynahate", "test"))

latenthatred_labels_train = read_labels("latenthatred", "train")
latenthatred_labels_dev = read_labels("latenthatred", "dev")
latenthatred_labels_test = read_labels("latenthatred", "test")

olid_labels_train = read_labels("olid", "train")
olid_labels_dev = read_labels("olid", "dev")
olid_labels_test = read_labels("olid", "test")

In [5]:
mlp = MLPClassifier(random_state=42)
gridsearch = GridSearchCV(
    mlp,
    param_grid={
        "hidden_layer_sizes": [(128), (128,64)],
        "activation": ["relu"],
        "solver": ["adam"],
        "learning_rate_init": [0.001, 0.0001],
        "learning_rate": ["adaptive"],
        "early_stopping": [True],
        "max_iter": [10000],
    },
    verbose=4,
    n_jobs=os.cpu_count()//3,
    cv=5,
)

In [6]:
dynahate_labels_train_dev = np.concatenate((dynahate_labels_train, dynahate_labels_dev))
latenthatred_labels_train_dev = np.concatenate((latenthatred_labels_train, latenthatred_labels_dev))
olid_labels_train_dev = np.concatenate((olid_labels_train, olid_labels_dev))

In [7]:
bert_dynahate_train_embeddings = reshape_embeddings(bert_dynahate_train_embeddings)
bert_dynahate_dev_embeddings = reshape_embeddings(bert_dynahate_dev_embeddings)
bert_dynahate_test_embeddings = reshape_embeddings(bert_dynahate_test_embeddings)

bert_latenthatred_train_embeddings = reshape_embeddings(bert_latenthatred_train_embeddings)
bert_latenthatred_dev_embeddings = reshape_embeddings(bert_latenthatred_dev_embeddings)
bert_latenthatred_test_embeddings = reshape_embeddings(bert_latenthatred_test_embeddings)

bert_olid_train_embeddings = reshape_embeddings(bert_olid_train_embeddings)
bert_olid_dev_embeddings = reshape_embeddings(bert_olid_dev_embeddings)
bert_olid_test_embeddings = reshape_embeddings(bert_olid_test_embeddings)

### BERT

#### DynaHate

In [8]:
bert_dynahate_train_dev = np.concatenate((bert_dynahate_train_embeddings, bert_dynahate_dev_embeddings))

grid_results = gridsearch.fit(bert_dynahate_train_dev, dynahate_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_dynahate_train_dev)
test_preds = mlp.predict(bert_dynahate_test_embeddings)

computeAllScores(train_dev_preds, test_preds, dynahate_labels_train_dev, dynahate_labels_test, "Results/bert_dynahate_seed_42")
save_model(mlp, "Saves/bert_dynahate_seed_42")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.6822601555747623
Accuracy Test:  0.6594660194174757
Weighted F1 Train Dev:  0.6799920440871515
Weighted F1 Test:  0.6567837440481107
Macro F1 Train Dev:  0.6768374922175435
Macro F1 Test:  0.6514448610162644
Micro F1 Train Dev:  0.6822601555747623
Micro F1 Test:  0.6594660194174757
Weighted Recall Train Dev:  0.6822601555747623
Weighted Recall Test:  0.6594660194174757
Macro Recall Train Dev:  0.6763393167843137
Macro Recall Test:  0.6507836515935868
Micro Recall Train Dev:  0.6822601555747623
Micro Recall Test:  0.6594660194174757
Confusion Matrix Train Dev: 
[[10232  6885]
 [ 4879 15028]]
Confusion Matrix Test: 
[[1046  806]
 [ 597 1671]]


#### LatentHatred

In [9]:
bert_latenthatred_train_dev = np.concatenate((bert_latenthatred_train_embeddings, bert_latenthatred_dev_embeddings))

grid_results = gridsearch.fit(bert_latenthatred_train_dev, latenthatred_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_latenthatred_train_dev)
test_preds = mlp.predict(bert_latenthatred_test_embeddings)

computeAllScores(train_dev_preds, test_preds, latenthatred_labels_train_dev, latenthatred_labels_test, "Results/bert_latenthatred_seed_42")
save_model(mlp, "Saves/bert_latenthatred_seed_42")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.6804469273743017
Accuracy Test:  0.6811918063314711
Weighted F1 Train Dev:  0.6459883320181068
Weighted F1 Test:  0.6466698995232357
Macro F1 Train Dev:  0.4287466975194995
Macro F1 Test:  0.43140895050642697
Micro F1 Train Dev:  0.6804469273743017
Micro F1 Test:  0.6811918063314711
Weighted Recall Train Dev:  0.6804469273743017
Weighted Recall Test:  0.6811918063314711
Macro Recall Train Dev:  0.4331882196943908
Macro Recall Test:  0.4329846786604845
Micro Recall Train Dev:  0.6804469273743017
Micro Recall Test:  0.6811918063314711
Confusion Matrix Train Dev: 
[[8776 1165    4]
 [3159 2178   10]
 [ 441  369    8]]
Confusion Matrix Test: 
[[2958  387    1]
 [1053  695    5]
 [ 144  122    5]]


#### OLID

In [10]:
bert_olid_train_dev = np.concatenate((bert_olid_train_embeddings, bert_olid_dev_embeddings))

grid_results = gridsearch.fit(bert_olid_train_dev, olid_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_olid_train_dev)
test_preds = mlp.predict(bert_olid_test_embeddings)

computeAllScores(train_dev_preds, test_preds, olid_labels_train_dev, olid_labels_test, "Results/bert_olid_seed_42")

save_model(mlp, "Saves/bert_olid_seed_42")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7505287009063444
Accuracy Test:  0.791860465116279
Weighted F1 Train Dev:  0.7427492612656574
Weighted F1 Test:  0.7679038497935156
Macro F1 Train Dev:  0.702817802300744
Macro F1 Test:  0.6887428600313401
Micro F1 Train Dev:  0.7505287009063443
Micro F1 Test:  0.791860465116279
Weighted Recall Train Dev:  0.7505287009063444
Weighted Recall Test:  0.791860465116279
Macro Recall Train Dev:  0.6942338543809132
Macro Recall Test:  0.6679435483870968
Micro Recall Train Dev:  0.7505287009063444
Micro Recall Test:  0.791860465116279
Confusion Matrix Train Dev: 
[[7621 1219]
 [2084 2316]]
Confusion Matrix Test: 
[[588  32]
 [147  93]]
